# 최종모델 저장

In [15]:
import pandas as pd
df_Y = pd.read_csv("../Data/뚝섬 한강공원 주차대수 예측.csv")

In [18]:
df_Y

,주차장명,월,휴일여부,최고기온,영동_아침주차,영동_낮주차,영동_저녁주차,영동_아침이용,영동_낮이용,영동_저녁이용,요일,주차구획수,연도
0,뚝섬2주차장,2,0,9.6,97.501291,222.281389,198.217320,10324.219759,23536.938585,20988.841656,3,356,2024
1,뚝섬1주차장,2,0,9.6,32.939625,75.095064,66.965311,1900.710504,4333.199742,3864.089755,3,67,2024
2,뚝섬3주차장,2,0,9.6,48.938872,111.569809,99.491319,3807.256027,8679.702038,7740.041936,3,112,2024
3,뚝섬4주차장,2,0,9.6,99.007103,225.714306,201.278591,10313.114399,23511.620821,20966.264780,3,136,2024
4,뚝섬3주차장,2,0,10.1,39.041615,84.172189,71.786196,3496.527013,7538.374930,6429.098057,2,112,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5935,뚝섬2주차장,1,0,5.5,23.588702,54.054480,49.356818,5755.457609,13188.867391,12042.675000,1,356,2020
5936,뚝섬3주차장,1,0,5.5,11.059134,25.347544,21.593322,279.910500,641.555069,546.534431,0,112,2020
5937,뚝섬3주차장,1,1,5.3,42.021679,117.752740,82.225582,2470.770520,6923.569136,4834.660344,6,112,2020
5938,뚝섬3주차장,1,1,5.9,47.470482,116.861358,88.668160,3413.183912,8402.470246,6375.345842,5,112,2020


In [17]:
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, KFold

# 1. 데이터 준비
features = ['월', '요일', '주차장명', '휴일여부', '연도', '최고기온']  # 입력 피처
target_usage = ['영동_아침이용', '영동_낮이용', '영동_저녁이용']  # 이용시간 타겟
parking_Target = ['영동_아침주차', '영동_낮주차', '영동_저녁주차']  # 주차대수 타겟

# 원본 데이터 복사
df_xgb = df_Y.copy()

# 범주형 데이터를 원-핫 인코딩하여 모델 입력으로 변환
X_xgb = pd.get_dummies(df_xgb[features])  # 이용시간 예측용 입력 피처
y_xgb_usage = df_xgb[target_usage]  # 이용시간 타겟
y_xgb_parking = df_xgb[parking_Target]  # 주차대수 타겟

# 원-핫 인코딩 결과를 float 타입으로 변환
X_xgb = X_xgb.astype(float)

# 2. 이용시간 예측 모델 학습 (KFold)
kf = KFold(n_splits=5, shuffle=True)
usage_train_scores = []
usage_test_scores = []

for train_index, test_index in kf.split(X_xgb):
    X_train_usage, X_test_usage = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
    y_train_usage, y_test_usage = y_xgb_usage.iloc[train_index], y_xgb_usage.iloc[test_index]

    # 이용시간 모델 학습
    usage_model = MultiOutputRegressor(
        XGBRegressor(learning_rate=0.1, max_depth=6, n_estimators=300)
    )
    usage_model.fit(X_train_usage, y_train_usage)

    # 점수 저장
    usage_train_scores.append(usage_model.score(X_train_usage, y_train_usage))
    usage_test_scores.append(usage_model.score(X_test_usage, y_test_usage))

# 평균 점수 출력
mean_usage_train_score = sum(usage_train_scores) / len(usage_train_scores)
mean_usage_test_score = sum(usage_test_scores) / len(usage_test_scores)
print(f"이용시간 모델 평균 훈련 점수 (R²): {mean_usage_train_score:.4f}")
print(f"이용시간 모델 평균 테스트 점수 (R²): {mean_usage_test_score:.4f}")

# 최종 이용시간 모델 학습 및 저장
usage_model.fit(X_xgb, y_xgb_usage)
joblib.dump(usage_model, 'Ttukseom_usage_model.pkl')
joblib.dump(X_xgb.columns.tolist(), 'Ttukseom_features_usage.pkl')  # 이용시간 피처 저장

# 예측된 이용시간 추가
df_xgb['예측 아침 이용시간'] = usage_model.predict(X_xgb)[:, 0]
df_xgb['예측 낮 이용시간'] = usage_model.predict(X_xgb)[:, 1]
df_xgb['예측 저녁 이용시간'] = usage_model.predict(X_xgb)[:, 2]

# 3. 주차대수 예측 모델 학습 (train_test_split)
features_with_usage = ['예측 아침 이용시간', '예측 낮 이용시간', '예측 저녁 이용시간'] + features
X_parking = pd.get_dummies(df_xgb[features_with_usage])

X_train_parking, X_test_parking, y_train_parking, y_test_parking = train_test_split(
    X_parking, y_xgb_parking, test_size=0.2
)

parking_model = MultiOutputRegressor(
    XGBRegressor(learning_rate=0.1, max_depth=6, n_estimators=300)
)
parking_model.fit(X_train_parking, y_train_parking)

# 모델 및 피처 순서 저장
joblib.dump(parking_model, 'Ttukseom_parking_model.pkl')
joblib.dump(X_parking.columns.tolist(), 'Ttukseom_features_parking.pkl')  # 주차대수 피처 저장

# 주차대수 모델 점수 출력
print(f"주차대수 모델 훈련 점수: {parking_model.score(X_train_parking, y_train_parking):.4f}")
print(f"주차대수 모델 테스트 점수: {parking_model.score(X_test_parking, y_test_parking):.4f}")

이용시간 모델 평균 훈련 점수 (R²): 0.9593
이용시간 모델 평균 테스트 점수 (R²): 0.8325
주차대수 모델 훈련 점수: 0.9842
주차대수 모델 테스트 점수: 0.8981
